# **データマイニングと情報可視化: 演習**
# **Data mining and information visualization: Exercise**
<font size="4">
第8回　2024年2月6日(火)<br>
データ選択・クロスバリデーション<br>
</font>

* [小テスト / Quiz](Week8_Quiz.ipynb)：授業中に提出してください。Please submit the quiz file by the end of the class.
* [冬学期授業アンケート](https://moodle.s.kyushu-u.ac.jp/mod/claque/response.php?id=1031733&cid=67159): 締め切り 2024年2月10日(火)

### 講義動画
* [解説動画](https://youtu.be/HHs-09bAXog)

In [1]:
# まずはライブラリをインポート / Import the libraries.
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 可視化ライブラリ / Visualization libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 機械学習ライブラリ / Machine learning library
import sklearn

# 小数第3位まで表示 / display up to the 3rd decimal place
%precision 3

'%.3f'

## アヤメのデータセットで解析 / Clustering analysis with a sample data set of iris.

Scikit-learnでは、いくつかサンプルデータが用意されていて、解析を試すのに使えるようになっています。<br>
今回は、その中でも、使われることの多い、アヤメのデータセットを使って、k-means法を試してみます。<br>
<br>
Scikit-learn provides several sample data sets to practise analysis data.<br>
This week, we use one of the most popular sample data, iris, and apply k

[Week5: アヤメのデータでk-means法クラスタリング](Ex_Week5.ipynb#iris)

[参考サイト: iris(アヤメ)のデータセットをpandasとseabornを使って可視化する](https://kenyu-life.com/2019/05/14/iris/)<br>
[参考サイト: 機械学習入門！クラスタリングの解説とPythonによるk-means実装](https://www.sejuku.net/blog/60630)<br>
[参考サイト: データサイエンティストによる統計入門 ― k平均法でデータをクラスタリングしてみよう！](https://employment.en-japan.com/engineerhub/entry/2018/04/10/110000)

In [ ]:
# scikit-learn のデータセットを読み込むためのインポート / Import 'datasets' to use datasets of scikit-learn.
from sklearn import datasets

#対象データを読み込み
iris = datasets.load_iris()
type(iris)

In [ ]:
# データの種類を確認 / Check the type of data
type(iris.data)

### DataFrame に変換 / Convert loaded data from Bunch to DataFrame

`feasure_names`を使って、`index`と`columns`を指定できます。<br>
You can set Index of the DataFrame with `feasure_names`.<br>

* sepal : 花のがく
* petal : 花弁、はなびら
* target: アヤメの種類 / types of iris

In [ ]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [ ]:
# 先頭の5行を表示


<img src="img/w5_iris.jpg" width="700px">

[参考サイト: iris(アヤメ)のデータセットをpandasとseabornを使って可視化する](https://kenyu-life.com/2019/05/14/iris/)<br>

In [ ]:
# データのサイズを確認します。 / Check the numbers of rows and columns of the data.


In [ ]:
# 目的変数をDataFrameに変換
df_target = pd.DataFrame(iris.target, columns=['species'])

# 説明変数のDataFrameに結合させて、iris_ansというDataFrameに格納
iris_ans = pd.concat([iris_df, df_target.species], axis=1)

In [ ]:
g_ans = iris_ans.groupby('species')

# Setosa
group00 = g_ans.get_group(0) 
#  Versicolor
group01 = g_ans.get_group(1)
# 2: Versinica
group02 = g_ans.get_group(2)


# 0th columns: sepal length: 花のがくの長さ
# 1st columns: sepal width: 花のがくの幅
# 2nd columns: petal length: 花弁の長さ
# 3rd columns: petal width: 花弁の幅
ax = group00.plot.scatter(x=group00.columns[1], y=group00.columns[2],color='red', label='Setosa')
ax = group01.plot.scatter(x=group01.columns[1], y=group01.columns[2],color='blue', label='Versicolor', ax=ax)
ax = group02.plot.scatter(x=group02.columns[1], y=group02.columns[2],color='green', label='Versinica', ax=ax)
plt.xlabel('sepal length (mm)')
plt.ylabel('sepal width (mm)')
# plt.savefig("w5_setalL_setalW.png", format="png")

### 欠損値の確認
データのレコード数や変数の数、欠損データを確認しておきましょう。<br>
Let's check the size of DataFrame, number of columns and NaN.<br>
<br>
欠損値の扱い方はこちら。/ How to deal with NaN.<br>
[Week 4: isnull](Ex_Week4.ipynb#isnull)<br>

In [ ]:
# use isnull function
# The output is the same size of an array with elements of True or False
# True : zero, NaN
# False :  nonzero



## k-meansクラスからirisデータセット向けに設定したインスタンスを作ります。
## Create an instance for iris data set from k-means class.

プログラムコードはこちら。 / Here is the code line.

```
kmeans = KMeans(n_clusters=3, max_iter=30, init="random", n_jobs=-1)
```

オプションには以下のような意味があります。
* n_clusters: クラスタの数 / number of clusters
* max_iter: 学習のループ回数 / number of iteration
* init: 平均の初期値の決め方 / how to determine initial position of the center of cluster.
* n_jobs: k-meansを何並列にするか(-1ならばpcのコア数分だけ並列してくれます）/how many cores in your CPU is used for the analysis

その他オプションの詳細はこちら<br>
[scikit-learn でクラスタ分析 (K-means 法)@Pythonでデータサイエンス](https://pythondatascience.plavox.info/scikit-learn/%E3%82%AF%E3%83%A9%E3%82%B9%E3%82%BF%E5%88%86%E6%9E%90-k-means)

In [ ]:
from sklearn.cluster import KMeans
# KMeansクラスの初期化 / Initialize KMeans class
# この例では 3 つのグループに分割 (メルセンヌツイスターの乱数の種を 10 とする)


# K-means クラスタリングをおこなう / Clustering with k-means method
# (1) fit 関数で学習する / Learning with 'fit' function


# (2) クラスター番号を予測 / Predict the cluster number.
y_pred = 


## クラスター番号をDataFrame 'iris_df'に追加する。/ Add a column 'cluster' to a DataFrame, 'iris_df'.

In [ ]:
# Copy data
# iris_df_k = pd.DataFrame(iris_df.values[:,0:4])
iris_df_k = iris_df
# iris_df_k.shape
# Add a column, cluster.
iris_df_k['cluster'] = y_pred
iris_df_k.tail(5)

## 可視化 : 散布図
Visualization : Scattering plot

In [ ]:
# クラスタリング結果のグラフ化 / Visualization
ax = None
bx = None
colors = ['blue', 'red', 'green']

# figsize : figureの縦横の大きさ(横、縦) / Size of figure (widthe, height)
# nrows : number of rows
# ncolumns : number of columns
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# LEFT FIGURE: Answer (iris.target)
for i, data in iris_ans.groupby('species'):
    #                       x-axis,  y-axis, specify the data with index name
    ax = data.plot.scatter(x=iris_ans.columns[0], y=iris_ans.columns[1], color=colors[i],
    #                                                           this number specify the position of the figure
                            title='iris.target', label=f'cluster{i}', ax=axes.flatten()[0])

# RIGHT FIGURE: k-means prediction
for i, data in iris_df_k.groupby('cluster'):
    #                       x-axis,  y-axis, specify the data with index name
    bx = data.plot.scatter(x=iris_df_k.columns[0], y=iris_df_k.columns[1], color=colors[i],
    #                                                           this number specify the position of the figure
                           title='prediction by k-means', label=f'cluster{i}', ax=axes.flatten()[1])


### クラスタの番号を付け替える

* 予測　クラスタ0 (red) → 答え　クラスタ1 (red)
* 予測　クラスタ1 (blue) → 答え　クラスタ2 (green)
* 予測　クラスタ2 (green) → 答え　クラスタ0 (blue)

クラスタ番号を入れ替えて、df_predに保存する。

[replace @ note.nkmk.me](https://note.nkmk.me/python-pandas-replace/)

In [ ]:
df_pred = pd.DataFrame(y_pred).replace([0, 1, 2], [1, 2, 0])

In [ ]:
iris_df_k['cluster_rep'] = df_pred

In [ ]:
# iris_ans.head()
iris_df_k.head()

In [ ]:
# クラスタリング結果のグラフ化 / Visualization
ax = None
bx = None
colors = ['blue', 'red', 'green']

# figsize : figureの縦横の大きさ(横、縦) / Size of figure (widthe, height)
# nrows : number of rows
# ncolumns : number of columns
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# LEFT FIGURE: Answer (iris.target)
for i, data in iris_ans.groupby('species'):
    #                       x-axis,  y-axis, specify the data with index name
    ax = data.plot.scatter(x=iris_ans.columns[0], y=iris_ans.columns[1], color=colors[i],
    #                                                           this number specify the position of the figure
                            title='iris.target', label=f'cluster{i}', ax=axes.flatten()[0])

# RIGHT FIGURE: k-means prediction
for i, data in iris_df_k.groupby('cluster_rep'):
    #                       x-axis,  y-axis, specify the data with index name
    bx = data.plot.scatter(x=iris_df_k.columns[0], y=iris_df_k.columns[1], color=colors[i],
    #                                                           this number specify the position of the figure
                           title='prediction by k-means', label=f'cluster{i}', ax=axes.flatten()[1])


### 正解率(accuracy)を計算

[scikit-learnで混同行列を生成、適合率・再現率・F1値などを算出](https://note.nkmk.me/python-sklearn-confusion-matrix-score/)

In [ ]:
from sklearn import metrics
# metrics.accuracy_score(iris_ans.species, df_pred)
metrics.accuracy_score(iris.target, df_pred)

<a id="svm"></a>

# サポートベクターマシンで分類

stratifyというオプションで、targetの割合が等しくなるように分割する。<br>
[Option: Stratify](https://panda-clip.com/data-split-stratified/)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm,metrics

# データを分割
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=0.4,random_state=0, stratify=iris.target)

# データを学習し、予測する
clf = svm.SVC()
clf.fit(X_train, y_train)
pred_svm = clf.predict(X_test)

## データを結合

In [ ]:
# Copy data
# iris_df_k = pd.DataFrame(iris_df.values[:,0:4])
iris_svm = pd.DataFrame(X_test, columns=iris.feature_names)

# Add a column, cluster.
iris_svm['cluster'] = pred_svm

iris_svm.head()

# # Answer
# iris_svm_ans = pd.DataFrame(X_test, columns=iris.feature_names)
# iris_svm_ans['cluster'] = y_test


## 可視化 : 散布図
Visualization : Scattering plot

In [ ]:
# クラスタリング結果のグラフ化 / Visualization
ax = None
bx = None
colors = ['blue', 'red', 'green']

# figsize : figureの縦横の大きさ(横、縦) / Size of figure (widthe, height)
# nrows : number of rows
# ncolumns : number of columns
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# LEFT FIGURE: Answer (iris.target)
for i, data in iris_svm_ans.groupby('cluster'):
    #                       x-axis,  y-axis, specify the data with index name
    ax = data.plot.scatter(x=iris_svm_ans.columns[0], y=iris_svm_ans.columns[1], color=colors[i],
    #                                                           this number specify the position of the figure
                            title='iris.target', label=f'cluster{i}', ax=axes.flatten()[0])

# RIGHT FIGURE: k-means prediction
for i, data in iris_svm.groupby('cluster'):
    #                       x-axis,  y-axis, specify the data with index name
    bx = data.plot.scatter(x=iris_svm.columns[0], y=iris_svm.columns[1], color=colors[i],
    #                                                           this number specify the position of the figure
                           title='prediction by SVM', label=f'cluster{i}', ax=axes.flatten()[1])


### クラスタの番号を付け替える(必要があれば。。。)

In [ ]:
# 今回は必要なかった。
# df_pred = pd.DataFrame(y_pred).replace([0, 1, 2], [2, 1, 0])

In [ ]:
#(4) 正解率を求める
ac_score = metrics.accuracy_score(y_test, pred_svm)
print('SVM 正解率：',ac_score) 

# k-means法の時は0.893だったので、正解率が上がってる!!
# データ数が少ないので、クロスバリデーションしたほうがいいかも。。

<a id="crossvalidation"></a>

## クロスバリデーションしてみる

In [ ]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC()

# 交差検証
scores = cross_val_score(clf, iris.data, iris.target)

# 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
# スコアの平均値

print('Average score: {}'.format(np.mean(scores)))

k-means法よりSVMのほうがうまく分類できるらしい。

<a id="tree"></a>

# 決定木で分類

今回はざっくりと使い方だけ紹介していますから、詳しくはきちんとテキストなどで勉強しましょう♪

* Web:[【機械学習】決定木をscikit-learnと数学の両方から理解する](https://qiita.com/Hawaii/items/53efe3e96b1171ebc7db)
* Book: [東京大学のデータサイエンティスト育成講座 : Pythonで手を動かして学ぶデータ分析](http://hdl.handle.net/2324/1001684368)
* Book: [Pythonによるデータマイニングと機械学習](http://hdl.handle.net/2324/1001680874)
* Book: [Pythonではじめる機械学習 : scikit-learnで学ぶ特徴量エンジニアリングと機械学習の基礎](http://hdl.handle.net/2324/1001632671)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import tree

# データを分割
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=0.4,random_state=0)

# データを学習し、予測する
clf = tree.DecisionTreeClassifier(max_depth=3) # 決定木モデル（最大深さ3）
clf = clf.fit(X_train, y_train) # 訓練データで学習

pred_tree = clf.predict(X_test)


## データを結合

In [ ]:
# Copy data
# iris_df_k = pd.DataFrame(iris_df.values[:,0:4])
iris_tree = pd.DataFrame(X_test, columns=iris.feature_names)

# Add a column, cluster.
iris_tree['cluster'] = pred_tree

iris_tree.head()

# Answer
iris_tree_ans = pd.DataFrame(X_test, columns=iris.feature_names)
iris_tree_ans['cluster'] = y_test


In [ ]:
iris_tree.head()

## 可視化 : 散布図
Visualization : Scattering plot

In [ ]:
# クラスタリング結果のグラフ化 / Visualization
ax = None
bx = None
colors = ['blue', 'red', 'green']

# figsize : figureの縦横の大きさ(横、縦) / Size of figure (widthe, height)
# nrows : number of rows
# ncolumns : number of columns
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# LEFT FIGURE: Answer (iris.target)
for i, data in iris_tree_ans.groupby('cluster'):
    #                       x-axis,  y-axis, specify the data with index name
    ax = data.plot.scatter(x=iris_tree_ans.columns[0], y=iris_tree_ans.columns[1], color=colors[i],
    #                                                           this number specify the position of the figure
                            title='iris.target', label=f'cluster{i}', ax=axes.flatten()[0])

# RIGHT FIGURE: k-means prediction
for i, data in iris_tree.groupby('cluster'):
    #                       x-axis,  y-axis, specify the data with index name
    bx = data.plot.scatter(x=iris_tree.columns[0], y=iris_tree.columns[1], color=colors[i],
    #                                                           this number specify the position of the figure
                           title='prediction by Decision Tree', label=f'cluster{i}', ax=axes.flatten()[1])


### クラスタの番号を付け替える(必要があれば。。。)

* 答え　クラスタ0 (blue) →予測　クラスタ2 (green)
* 答え　クラスタ1 (red) →予測　クラスタ1 (red)
* 答え　クラスタ2 (green) →予測　クラスタ0 (blue)

クラスタ番号を入れ替えて、df_predに保存する。

In [ ]:
# 今回は必要なかった。
# df_pred = pd.DataFrame(y_pred).replace([0, 1, 2], [2, 1, 0])

In [ ]:
#(4) 正解率を求める
ac_score = metrics.accuracy_score(y_test, pred_tree)
print('Decision Tree 正解率：',ac_score) 

# k-means法 -> 0.893
# SVM -> 0.97
# さらに正解率が上がってる!!

## クロスバリデーションしてみる

In [ ]:
from sklearn.model_selection import cross_val_score
clf = tree.DecisionTreeClassifier(max_depth=3) # 決定木モデル（最大深さ3）

# 交差検証
scores = cross_val_score(clf, iris.data, iris.target)

# 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
# スコアの平均値

print('Average score: {}'.format(np.mean(scores)))

クロスバリデーションすると、SVMと決定木はだいたい同じ正解率だった。